In [15]:
import torch

torch.cuda.is_available()

True

# 데이터셋 선택 및 하이퍼파라미터 설정
▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼

In [1]:
import time
import natsort
import os

folder_list = os.listdir("./data/")
item_list = natsort.natsorted(folder_list)

print("다음 데이터셋들이 학습됩니다 : ", item_list)

다음 데이터셋들이 학습됩니다 :  ['bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather', 'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper']


In [ ]:
#최소10, 200~400 추천, 10단위로 pth가 저장됨
epochs = 200
batch_size = 16

learning_rate = 0.005
image_size = 256

▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲

# Main.py

In [ ]:
import torch
from dataset import get_data_transforms
from torchvision.datasets import ImageFolder
import numpy as np
import random
import os
from torch.utils.data import DataLoader
from resnet import  wide_resnet50_2
from de_resnet import de_wide_resnet50_2
from dataset import RD_Dataset
import torch.backends.cudnn as cudnn
import argparse
from torch.nn import functional as F

In [4]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
def loss_fucntion(a, b):
    cos_loss = torch.nn.CosineSimilarity()
    loss = 0
    for item in range(len(a)):
        loss += torch.mean(1-cos_loss(a[item].view(a[item].shape[0],-1),
                                      b[item].view(b[item].shape[0],-1)))
    return loss

In [ ]:
def loss_concat(a, b):
    mse_loss = torch.nn.MSELoss()
    cos_loss = torch.nn.CosineSimilarity()
    loss = 0
    a_map = []
    b_map = []
    size = a[0].shape[-1]
    for item in range(len(a)):
        a_map.append(F.interpolate(a[item], size=size, mode='bilinear', align_corners=True))
        b_map.append(F.interpolate(b[item], size=size, mode='bilinear', align_corners=True))
    a_map = torch.cat(a_map,1)
    b_map = torch.cat(b_map,1)
    loss += torch.mean(1-cos_loss(a_map,b_map))
    return loss

In [ ]:
def train(_class_):
    print(_class_)
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)

    data_transform = get_data_transforms(image_size, image_size)
    
    train_path = './data/' + _class_ + '/train'
    ckp_path = './checkpoints/' + 'wres50_'+_class_+'.pth'
    
    train_data = ImageFolder(root=train_path, transform=data_transform)
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

    encoder, bn = wide_resnet50_2(pretrained=True)
    encoder = encoder.to(device)
    bn = bn.to(device)
    encoder.eval()
    decoder = de_wide_resnet50_2(pretrained=False)
    decoder = decoder.to(device)

    optimizer = torch.optim.Adam(list(decoder.parameters())+list(bn.parameters()), lr=learning_rate, betas=(0.5,0.999))


    for epoch in range(epochs):
        start = time.time() 
        
        bn.train()
        decoder.train()
        loss_list = []
        for img, label in train_dataloader:
            img = img.to(device)
            inputs = encoder(img)
            outputs = decoder(bn(inputs))
            loss = loss_fucntion(inputs, outputs)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, epochs, np.mean(loss_list)))
        print("time :",time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
        
        if (epoch + 1) % 10 == 0:
            torch.save({'bn': bn.state_dict(),'decoder': decoder.state_dict()}, ckp_path)
            
    return loss

# 학습 시작

In [12]:
setup_seed(111)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
#학습
for i in item_list:
    start_class = time.time()  # 시작 시간 저장

    train(i)
    print(i, "time :",time.time() - start_class)  # 현재시각 - 시작시간 = 실행 시간

hazelnut
cuda
epoch [1/10], loss:0.9380
time : 57.73496961593628
epoch [2/10], loss:0.5681
time : 38.83104968070984
epoch [3/10], loss:0.4682
time : 45.24616718292236
epoch [4/10], loss:0.4185
time : 46.11638784408569
epoch [5/10], loss:0.3803
time : 39.50839972496033
epoch [6/10], loss:0.3532
time : 49.41082715988159
epoch [7/10], loss:0.3326
time : 49.57550024986267
epoch [8/10], loss:0.3167
time : 47.884011030197144
epoch [9/10], loss:0.3033
time : 61.98079538345337
epoch [10/10], loss:0.2911
time : 50.9464898109436
hazelnut time : 489.9562199115753
